In [0]:
%sql
USE CATALOG `100cep_gateway`;

In [0]:
%sql
USE SCHEMA silver; 

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_pedidos AS
WITH bronze_order_normalized AS (
  SELECT 
    TRIM(LOWER(order_id)) AS pedido_id,
    TRIM(LOWER(customer_id)) AS cliente_id,
    TRIM(
      UPPER(
        TRANSLATE(
          AI_TRANSLATE(order_status, 
            'portuguese'),
            'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ', 
            'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
        )
      )
    ) AS status_pedido,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_purchase_timestamp :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS data_pedido,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_approved_at :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS aprovacao_pedido,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_delivered_carrier_date :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS data_entrega_transportadora,
    DATE_FORMAT(
      FROM_UTC_TIMESTAMP(order_delivered_customer_date :: TIMESTAMP, 'America/Sao_Paulo'),
      'yyyy-MM-dd HH:mm:ss'
    ) AS data_entrega_cliente,
    order_estimated_delivery_date :: DATE AS data_entrega_estimada
  FROM 
    100cep_gateway.bronze.100cep_orders_raw
)
SELECT
  *
FROM 
  bronze_order_normalized;


In [0]:
%sql
SELECT
  *
FROM
  100cep_pedidos
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_pagamentos AS
WITH bronze_order_payments_normalized AS (
  SELECT 
    order_id AS pedido_id,
    CAST(payment_sequential AS INT) AS pagamento_sequencial,
    TRIM(
      UPPER(
        TRANSLATE(
          AI_TRANSLATE(
            REPLACE(payment_type, '_', ' '), 
            'portuguese'),
          'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
          'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
        )
      )
    ) AS tipo_pagamento,
    CAST(payment_installments AS INT) AS pagamento_parcelas,
    CAST(ROUND(payment_value, 2) AS FLOAT) AS valor_transacao
  FROM 
    100cep_gateway.bronze.100cep_order_payments_raw
)
SELECT 
  *
FROM 
  bronze_order_payments_normalized;

In [0]:
%sql
SELECT
  *
FROM
  100cep_pagamentos
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_itens_pedidos AS
WITH bronze_order_items_normalized AS (
SELECT 
  order_id AS pedido_id,
  order_item_id AS item_pedido_id,
  product_id AS produto_id,
  seller_id AS vendedor_id,
  CAST(shipping_limit_date AS DATE) AS data_entrega_limite,
  price :: DECIMAL (6, 2) AS preco_produto,
  freight_value :: DECIMAL(5, 2) AS valor_frete
FROM 
  100cep_gateway.bronze.`100cep_order_items_raw`
) 
SELECT 
  * 
FROM 
  bronze_order_items_normalized;

In [0]:
%sql
SELECT
  *
FROM
  100cep_itens_pedidos
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_clientes AS
WITH bronze_customers_normalized AS (
  SELECT 
    customer_id AS cliente_id,
    customer_unique_id AS cliente_id_unico,
    customer_zip_code_prefix AS cep_prefixo
  FROM 
    100cep_gateway.bronze.100cep_customers_raw
)
SELECT 
  *
FROM 
  bronze_customers_normalized;

In [0]:
%sql
SELECT 
  *
FROM 
  100cep_clientes
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_vendedores AS
WITH bronze_sellers_normalized AS (
SELECT 
  seller_id AS vendedor_id,
  seller_zip_code_prefix AS cep_prefixo
FROM 
  `100cep_gateway`.bronze.`100cep_sellers_raw`
)
SELECT 
  *
FROM
  bronze_sellers_normalized;

In [0]:
%sql
SELECT 
  *
FROM 
  100cep_vendedores
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_avaliacoes AS
WITH bronze_order_reviews_normalized AS (
SELECT
   TRIM(LOWER(
       CASE WHEN LENGTH(review_id) = 32 AND review_id NOT LIKE '% %' 
            THEN review_id END
   )) AS avaliacao_id,

   TRIM(LOWER(
       CASE WHEN LENGTH(order_id) = 32 AND order_id NOT LIKE '% %' 
            THEN order_id END
   )) AS pedido_id,

   review_score AS avaliacao_nota,

   TRIM(UPPER(
        CASE 
            WHEN AI_ANALYZE_SENTIMENT(review_comment_message) = 'positive' THEN 'positivo'
            WHEN AI_ANALYZE_SENTIMENT(review_comment_message) = 'negative' THEN 'negativo'
            WHEN AI_ANALYZE_SENTIMENT(review_comment_message) = 'neutral' THEN 'neutro'
            ELSE 'desconhecido'
        END
   )) AS `avaliacao_sentimento`,

   TRY_CAST(
       CASE WHEN LENGTH(review_creation_date) = 19 
            THEN review_creation_date END
       AS DATE
   ) AS avaliacao_data,

   review_answer_timestamp AS avaliacao_hora_resposta
FROM
  `100cep_gateway`.bronze.`100cep_order_reviews_raw`
WHERE 
    (LENGTH(review_id) = 32 AND review_id NOT LIKE '% %')
 OR (LENGTH(order_id) = 32 AND order_id NOT LIKE '% %')
)
SELECT 
  *
FROM 
  bronze_order_reviews_normalized;

In [0]:
%sql
SELECT 
  *
FROM 
  100cep_avaliacoes
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_categoria_produtos AS
WITH bronze_product_category_normalized AS (
SELECT 
    TRIM(UPPER(TRANSLATE(
        product_category_name,
        'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
        'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
      ))) AS categoria_produto,
    TRIM(UPPER(TRANSLATE(
        product_category_name_english,
        'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
        'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
      ))) AS categoria_produto_ingles
FROM `100cep_gateway`.bronze.`100cep_product_category_name_translation_raw`
)
SELECT 
    *
FROM 
    bronze_product_category_normalized;

In [0]:
%sql
SELECT 
  *
FROM 
  100cep_categoria_produtos
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_produtos AS 
WITH bronze_products_normalized AS (
SELECT 
  product_id AS produto_id,
  TRIM(UPPER(TRANSLATE(
        product_category_name,
        'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
        'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
      ))) AS categoria_produto,
  product_name_lenght AS nome_produto_tamanho,
  product_description_lenght AS descricao_produto_tamanho,
  product_photos_qty AS fotos_produto_quantidade,
  product_weight_g AS peso_produto,
  product_length_cm AS comprimento_produto,
  product_height_cm AS altura_produto,
  product_width_cm AS largura_produto
FROM 
  `100cep_gateway`.bronze.`100cep_products_raw`
)
SELECT 
  *
FROM 
  bronze_products_normalized;

In [0]:
%sql
SELECT 
  *
FROM 
  100cep_produtos
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_geolocalizacao AS
WITH geolocation_normalized AS (
SELECT
geolocation_zip_code_prefix AS cep_prefixo,
  geolocation_lat AS latitude,
  geolocation_lng AS longitude,
  TRIM(
    UPPER(
      REGEXP_REPLACE(
        TRANSLATE(
          geolocation_city,
          'áéíóúàèìòùãõâêîôûäëïöüçÁÉÍÓÚÀÈÌÒÙÃÕÂÊÎÔÛÄËÏÖÜÇ',
          'aeiouaeiouaoaeiouaeioucAEIOUAEIOUAOAEIOUAEIOUC'
        ),
        '[^A-Za-z0-9 ]',
        ''
      )
    )
  ) AS cidade,
  TRIM(UPPER(geolocation_state)) AS estado
FROM 
  `100cep_gateway`.bronze.`100cep_geolocation_raw`
ORDER BY
  cidade
)
SELECT 
  *
FROM 
  geolocation_normalized;

In [0]:
%sql
SELECT 
  *
FROM 
  100cep_geolocalizacao
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE 100cep_chargebacks
WITH bronze_chargebacks_normalized AS (
SELECT 
  chargeback_id,
  TRIM(UPPER(chargeback_reason)) AS motivo_chargeback,
  TRIM(UPPER(chargeback_status)) AS status_chargeback,
  chargeback_amount :: DECIMAL(6,2) AS valor_chargeback,
  TRIM(UPPER(issuer_response)) AS resposta_emitente,
  TRIM(UPPER(REPLACE(acquirer_response, "disptutando", "disputando"))) AS resposta_adquirente
FROM `100cep_gateway`.bronze.`100cep_chargebacks_raw`
WHERE chargeback_id IS NOT NULL
)
SELECT 
  *
FROM 
  bronze_chargebacks_normalized;

In [0]:
%sql
SELECT 
  * 
FROM 
  100cep_chargebacks
LIMIT 10

In [0]:
%sql
SHOW TABLES

In [0]:
%sql
SELECT 'avaliacoes' AS table_name, COUNT(*) AS n_linhas FROM 100cep_avaliacoes
UNION ALL
SELECT 'categoria_produtos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_categoria_produtos
UNION ALL
SELECT 'clientes' AS table_name, COUNT(*) AS n_linhas FROM 100cep_clientes
UNION ALL
SELECT 'geolocalizacao' AS table_name, COUNT(*) AS n_linhas FROM 100cep_geolocalizacao
UNION ALL
SELECT 'itens_pedidos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_itens_pedidos
UNION ALL
SELECT 'pagamentos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_pagamentos
UNION ALL
SELECT 'pedidos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_pedidos
UNION ALL
SELECT 'produtos' AS table_name, COUNT(*) AS n_linhas FROM 100cep_produtos
UNION ALL
SELECT 'vendedores' AS table_name, COUNT(*) AS n_linhas FROM 100cep_vendedores
UNION ALL
SELECT 'chargebacks' AS table_name, COUNT(*) AS n_linhas FROM 100cep_chargebacks;